In [2]:
import pandas as pd

In [6]:
df=pd.read_csv("Raw_Data.csv", encoding='latin-1')

In [8]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [18]:
df.shape

(406829, 8)

In [24]:
df.describe

<bound method NDFrame.describe of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

            InvoiceDate  UnitPrice  Custo

In [40]:
#Handling missing data 
df=df.dropna(subset=["CustomerID"])

In [42]:
df.shape

(406829, 8)

In [44]:
#Filling missing values
df["UnitPrice"]=df["UnitPrice"].fillna(0)
df["Quantity"]=df["Quantity"].fillna(0)

In [48]:
#Removing orders when quantity is 0
df=df[df["Quantity"]>0]

In [50]:
#Removing invoice data that starts with C
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [52]:
# Changing Date to datetime format 
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])

In [54]:
#Create new feature total purchase 
df["TotalPurchase"]=df['Quantity'] * df['UnitPrice']

In [56]:
#Creating snapshot to use recency
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

In [58]:
# Group by customer
customer_df = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,# Recency
    'InvoiceNo': 'nunique',# Frequency
    'TotalPurchase': 'sum'# Monetary Value
})

In [60]:
customer_df.head()

,InvoiceDate,InvoiceNo,TotalPurchase
CustomerID,,,
12346.0,326,1,77183.60
12347.0,2,7,4310.00
12348.0,75,4,1797.24
12349.0,19,1,1757.55
12350.0,310,1,334.40


In [62]:
# Rename columns
customer_df.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPurchase': 'MonetaryValue'
}, inplace=True)

In [64]:
customer_df.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,1,77183.60
12347.0,2,7,4310.00
12348.0,75,4,1797.24
12349.0,19,1,1757.55
12350.0,310,1,334.40


In [66]:
# Reset index
customer_df = customer_df.reset_index()